In [ ]:
# Load appropriate packages
from simple_unet_model import simple_unet_model   #Use normal unet model
from tensorflow.keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
image_directory = 'images/'
mask_directory = 'mask/'

In [ ]:
def make_tiles(dir_in: str, dir_out: str, d: int):
    """tiles all image files in a given folder
    Args:
        dir_in (str): directory with image files
        dir_out (str): output directory
        d (int): tilesize in pixels
    """
    for filename in glob.glob(
        dir_in + "/*.tif"
    ):
        img = cv2.imread(filename)
        name, ext = os.path.splitext(filename)
        name = name.split("/")[1]
        w, h = img.size
        grid = product(range(0, h - h % d, d), range(0, w - w % d, d))
        
        for i, j in grid:
            box = (j, i, j + d, i + d)
            cropped = img.crop(box)
            output = os.path.join(dir_out, f"{name}_{i}_{j}{ext}")
            cv2.imwrite(output, cropped)

In [ ]:
import os
import gdal

in_path = 'images/'
input_filename = 'test_1.tif'

out_path = 'mask/'
output_filename = 'test_1_tile_'

tile_size_x = 50
tile_size_y = 70

ds = gdal.Open(in_path + input_filename)
band = ds.GetRasterBand(1)
xsize = band.XSize
ysize = band.YSize

for i in range(0, xsize, tile_size_x):
    for j in range(0, ysize, tile_size_y):
        com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(in_path) + str(input_filename) + " " + str(out_path) + str(output_filename) + str(i) + "_" + str(j) + ".tif"
        os.system(com_string)

In [ ]:
def make_tiles(dir_in: str, dir_out: str, d: int):
    SIZE = d
    images = os.listdir(dir_in)
    image_dataset = []
    mask_dataset = []

    for i, image_name in enumerate(images):  
        if (image_name.split('.')[1] == 'tif'):
            image = cv2.imread(image_directory+image_name, 0)
            image = Image.fromarray(image)
            image = image.resize((SIZE, SIZE))
            image_dataset.append(np.array(image))


        name, ext = os.path.splitext(filename)
        name = name.split("/")[1]
        w, h = img.size
        grid = product(range(0, h - h % d, d), range(0, w - w % d, d))
        
        for i, j in grid:
            box = (j, i, j + d, i + d)
            cropped = img.crop(box)
            output = os.path.join(dir_out, f"{name}_{i}_{j}{ext}")
            cv2.imwrite(output, cropped)




#Iterate through all images in Uninfected folder, resize to 64 x 64
#Then save into the same numpy array 'dataset' but with label 1

masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))


#Normalize images
image_dataset = np.expand_dims(normalize(np.array(image_dataset), axis=1),3)
#D not normalize masks, just rescale to 0 to 1.
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.10, random_state = 0)

#Sanity check, view few mages
import random
import numpy as np
image_number = random.randint(0, len(X_train))
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(np.reshape(X_train[image_number], (256, 256)), cmap='gray')
plt.subplot(122)
plt.imshow(np.reshape(y_train[image_number], (256, 256)), cmap='gray')
plt.show()